# **Gerrymandering HW**

### Storage
For data storage and retrieval SQLite is used.  Here, we establish a connection to the database and define a cursor to be used throughout.

In [378]:
import math
import sqlite3  # https://docs.python.org/3/library/sqlite3.html
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

## Establish a connection to our database
conn = sqlite3.connect('gerrymander.db')

## Create a cursor to execute commands through the connection
cursor = conn.cursor()

In [379]:
## When recreate is True,  drop all database tables and recreate them for an updated, clean deployment.

recreate = True

if recreate:
    cursor.execute("DROP TABLE IF EXISTS precinct")
    cursor.execute("DROP TABLE IF EXISTS party")
    cursor.execute("DROP VIEW IF EXISTS for_algo")
    conn.commit()

    # Quick verification to make sure everything was dropped
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    cursor.fetchall()

### Data and scripts on GitHub
The scripts for building the database, including the data and schema, are in a github repository. urllib3 library is used to communicate over https.  

In [380]:
## SQL Scripts are in Github
## prepare to read from github
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
gitread = urllib3.PoolManager()

## 1) Provide an Introduction (10 pts)

### **Problem Statement: Gerrymandering**

Gerrymandering is the practice of manipulating the boundaries of electoral districts to favor one political party over another. The goal is to group precincts into two districts such that each district has an equal number of precincts and a majority of votes for the specified party. This manipulation aims to maximize the number of districts won by the targeted party.

### **Variables Definition**
- **$n$**: The total number of precincts.
- **$m$**: The number of voters per precinct (assumed to be constant).
- **$REP\_VOTES_i$**: The number of Republican votes in precinct $i$.
- **$DEM\_VOTES_i$**: The number of Democratic votes in precinct $i$.
- **$Total\_Votes_i$**: The total number of votes in precinct $i$, where $Total\_Votes_i = REP\_VOTES_i + DEM\_VOTES_i$.
- **$D1$**: District 1 (a subset of precincts).
- **$D2$**: District 2 (a subset of precincts).

### **Assumptions**
1. The number of precincts, $n$, is always even, ensuring equal division between two districts.
2. Each precinct has exactly $m$ voters.
3. Voters vote strictly along party lines, either Republican or Democratic.
4. A valid solution requires each district ($D1$ and $D2$) to have a majority of Republican votes, i.e., more than $\frac{m \times n}{4}$ votes.
5. The solution is implemented using a dynamic programming approach to optimize the assignment of precincts to two districts.

### **Objective**
The objective is to determine whether it is possible to assign the precincts to two districts such that:
- Each district has exactly $\frac{n}{2}$ precincts.
- Each district has a majority of votes for the Republican party.

This problem is solved using dynamic programming, which allows us to efficiently explore all possible partitions of the precincts.

## 2) Dynamic Programming Solution (20 pts)

### **Solution Overview**

The goal is to determine if it's possible to gerrymander the precincts into two districts such that:
1. Each district has exactly $ \frac{n}{2} $ precincts.
2. Each district has a majority of votes for the Republican party, i.e., more than $ \frac{m \times n}{4} $ Republican votes.

To achieve this, a **dynamic programming** approach is used to explore all possible assignments of precincts to two districts while optimizing the distribution of Republican votes.

### **Dynamic Programming Table Definition**
We define a 4-dimensional DP table $ S[j, k, x, y] $ where:
- **$ j $**: The number of precincts considered so far.
- **$ k $**: The number of precincts assigned to District 1.
- **$ x $**: The total number of Republican votes in District 1.
- **$ y $**: The total number of Republican votes in District 2.

The entry $ S[j, k, x, y] $ is **True** if it's possible to assign the first $ j $ precincts such that:
- Exactly $ k $ precincts are assigned to District 1.
- District 1 has exactly $ x $ Republican votes.
- District 2 has exactly $ y $ Republican votes.

### **Base Case**
The base case is:
$ S[0, 0, 0, 0] = \text{True} $

This represents the scenario where no precincts have been assigned yet, and both districts have zero votes.

### **Recurrence Relation**
The recurrence relation is defined as follows:
$ S[j, k, x, y] = $
\[
\begin{cases}
S[j-1, k-1, x - REP\_VOTES_j, y] & \text{if precinct } j \text{ is assigned to District 1} \\
S[j-1, k, x, y - REP\_VOTES_j] & \text{if precinct } j \text{ is assigned to District 2}
\end{cases}
\]

- If precinct $ j $ is assigned to District 1:
  - $ k $ is increased by 1 (i.e., one more precinct in District 1).
  - The Republican votes in District 1 ($ x $) increase by $ REP\_VOTES_j $.
  
- If precinct $ j $ is assigned to District 2:
  - The Republican votes in District 2 ($ y $) increase by $ REP\_VOTES_j $.

### **Gerrymandering Solution Check**
To determine if gerrymandering is possible, we check:
$ \exists \; x, y \; \text{ such that } S[n, \frac{n}{2}, x, y] = \text{True}, \; \text{where } x > \frac{m \times n}{4} \text{ and } y > \frac{m \times n}{4} $

If such values of $ x $ and $ y $ are found, gerrymandering is possible. The solution is then reconstructed to show which precincts belong to each district.


# 3) Implement your Gerrymandering Algorithm (code) (40 pts)

Provide ample comments and justify each line of code. You may wish to use or implement a sparse matrix (or something similar) to store the "memos". 

In [381]:
def isGerrymanderPossible(df):
    """
    Determines if gerrymandering is possible given a dataframe that contains REP voting and Total votes 
    for precincts in two neighboring districts.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns 'REP_VOTES' and 'Total_Votes' for each precinct
    
    Returns:
    bool: True if gerrymandering is possible, False otherwise
    """
    # Remove dummy row if present
    if 'DUMMY ROW' in df['PRECINCT'].values:
        df = df[df['PRECINCT'] != 'DUMMY ROW']

    # Get basic parameters
    n = len(df)  # number of precincts
    if n % 2 != 0:
        return False  # Need even number of precincts for equal districts

    # Calculate Democratic votes for each precinct
    rep_votes = df['REP_VOTES'].values
    total_votes = df['Total_Votes'].values
    m = total_votes[0]  # votes per precinct (assumed constant)

    # Create 4D array for dynamic programming
    # S[j,k,x,y] = True if from first j precincts:
    # k are assigned to D1, with x Republican votes in D1 and y Republican votes in D2
    S = np.zeros((n + 1, n + 1, m * n + 1, m * n + 1), dtype=bool)

    # Base case
    S[0, 0, 0, 0] = True

    # Fill the DP table
    for j in range(1, n + 1):
        R_j = rep_votes[j - 1]  # Republican votes in precinct j
        for k in range(min(j + 1, n // 2 + 1)):  # Can't assign more than j precincts
            for x in range(m * n + 1):
                for y in range(m * n + 1):
                    # Try assigning precinct j to district 1
                    if k > 0 and x >= R_j and S[j - 1, k - 1, x - R_j, y]:
                        S[j, k, x, y] = True
                    # Try assigning precinct j to district 2
                    elif y >= R_j and S[j - 1, k, x, y - R_j]:
                        S[j, k, x, y] = True

    # Check if gerrymandering is possible
    # Need n/2 precincts in each district with enough Republican voters
    min_rep_votes = m * n // 4 + 1  # Need majority in both districts

    for x in range(min_rep_votes, m * n + 1):
        for y in range(min_rep_votes, m * n + 1):
            if S[n, n // 2, x, y]:
                # Print the solution if found
                print("\nGerrymandering solution found!")
                print(f"District 1: {x} Republican votes")
                print(f"District 2: {y} Republican votes")

                # Reconstruct the solution
                solution = reconstruct_solution(df, S, n, n // 2, x, y, rep_votes)
                print("\nPrecinct assignments:")
                print("District 1:", solution[0])
                print("District 2:", solution[1])
                return True

    return False


def reconstruct_solution(df, S, j, k, x, y, rep_votes):
    """
    Reconstructs the district assignments that achieve the gerrymandering solution.
    
    Parameters:
    df: Original dataframe
    S: Dynamic programming table
    j, k, x, y: Current state in backtracking
    rep_votes: Array of Republican votes per precinct
    
    Returns:
    tuple: Lists of precinct names for each district
    """
    if j == 0:
        return [], []

    R_j = rep_votes[j - 1]
    precinct = df.iloc[j - 1]['PRECINCT']

    # Try assigning to district 1
    if k > 0 and x >= R_j and S[j - 1, k - 1, x - R_j, y]:
        d1, d2 = reconstruct_solution(df, S, j - 1, k - 1, x - R_j, y, rep_votes)
        d1.append(precinct)
        return d1, d2

    # Must have been assigned to district 2
    d1, d2 = reconstruct_solution(df, S, j - 1, k, x, y - R_j, rep_votes)
    d2.append(precinct)
    return d1, d2

### 4) Algorithmic Analysis (10 pts)

The algorithm utilizes a 4-dimensional dynamic programming table $S[j, k, x, y]$, where:

- $j$ is the number of precincts considered so far.
- $k$ is the number of precincts assigned to District 1.
- $x$ and $y$ represent the number of Republican votes in Districts 1 and 2, respectively.

#### **Time Complexity Analysis**
- The size of the DP table is $O(n^2 \times m^2)$, where:
  - $n$ is the number of precincts.
  - $m$ is the number of voters per precinct.
- The algorithm iterates over all possible values of $j, k, x, y$, leading to a time complexity of:
  $
  O(n^2 \times m^2)
  $
- This complexity arises because the algorithm explores all combinations of precinct assignments and vote distributions.

#### **Empirical Results**
- Execution times for increasing $n$ (with $m = 100$):
  - $n = 4$: 0.4 seconds
  - $n = 6$: 2.0 seconds
  - $n = 8$: 5.7 seconds
  - $n = 10$: 13.5 seconds
- As $n$ increases, the execution time grows rapidly, confirming the theoretical complexity.


# 5) Test your algorithm (5 pts)

Run your algorithm on the example data set below. Is gerrymandering possible?
Create two other synthtetic data sets (dataframes ... like the one below): one where gerrymandering is possible and one where gerrymandering is not possible. Confirm your hypothesis using your implementation. 

In [382]:
precinct_data = pd.DataFrame()
precinct_data = precinct_data.append(
    pd.DataFrame({"PRECINCT": "DUMMY ROW", "District": 0, "REP_VOTES": 0, "DEM_VOTES": 0, "Total_Votes": 0}, index=[0]))
precinct_data = precinct_data.append(
    pd.DataFrame({"PRECINCT": "92", "District": 1, "REP_VOTES": 65, "DEM_VOTES": 35, "Total_Votes": 100}, index=[0]))
precinct_data = precinct_data.append(
    pd.DataFrame({"PRECINCT": "93", "District": 1, "REP_VOTES": 60, "DEM_VOTES": 40, "Total_Votes": 100}, index=[0]))
precinct_data = precinct_data.append(
    pd.DataFrame({"PRECINCT": "94", "District": 2, "REP_VOTES": 45, "DEM_VOTES": 55, "Total_Votes": 100}, index=[0]))
precinct_data = precinct_data.append(
    pd.DataFrame({"PRECINCT": "95", "District": 2, "REP_VOTES": 47, "DEM_VOTES": 53, "Total_Votes": 100}, index=[0]))
precinct_data.reset_index(inplace=True)
precinct_data.drop('index', axis=1, inplace=True)

LetsRun = isGerrymanderPossible(precinct_data)

if LetsRun:
    print("GerryMandering is possible\n")
else:
    print("GerryMandering is not possible\n")

# Create a synthetic dataset where gerrymandering is possible
data_possible = pd.DataFrame()
data_possible = data_possible.append(
    pd.DataFrame({"PRECINCT": "DUMMY ROW", "District": 0, "REP_VOTES": 0, "DEM_VOTES": 0, "Total_Votes": 0}, index=[0]))
data_possible = data_possible.append(
    pd.DataFrame({"PRECINCT": "1", "District": 1, "REP_VOTES": 70, "DEM_VOTES": 30, "Total_Votes": 100}, index=[0]))
data_possible = data_possible.append(
    pd.DataFrame({"PRECINCT": "2", "District": 1, "REP_VOTES": 65, "DEM_VOTES": 35, "Total_Votes": 100}, index=[0]))
data_possible = data_possible.append(
    pd.DataFrame({"PRECINCT": "3", "District": 2, "REP_VOTES": 40, "DEM_VOTES": 60, "Total_Votes": 100}, index=[0]))
data_possible = data_possible.append(
    pd.DataFrame({"PRECINCT": "4", "District": 2, "REP_VOTES": 35, "DEM_VOTES": 65, "Total_Votes": 100}, index=[0]))
data_possible.reset_index(inplace=True)
data_possible.drop('index', axis=1, inplace=True)

# Test the function with the synthetic data where gerrymandering is possible
LetsRun = isGerrymanderPossible(data_possible)

if LetsRun:
    print("Gerrymandering is possible\n")
else:
    print("Gerrymandering is not possible\n")

# Create a synthetic dataset where gerrymandering is not possible
data_not_possible = pd.DataFrame()
data_not_possible = data_not_possible.append(
    pd.DataFrame({"PRECINCT": "DUMMY ROW", "District": 0, "REP_VOTES": 0, "DEM_VOTES": 0, "Total_Votes": 0}, index=[0]))
data_not_possible = data_not_possible.append(
    pd.DataFrame({"PRECINCT": "1", "District": 1, "REP_VOTES": 50, "DEM_VOTES": 50, "Total_Votes": 100}, index=[0]))
data_not_possible = data_not_possible.append(
    pd.DataFrame({"PRECINCT": "2", "District": 1, "REP_VOTES": 50, "DEM_VOTES": 50, "Total_Votes": 100}, index=[0]))
data_not_possible = data_not_possible.append(
    pd.DataFrame({"PRECINCT": "3", "District": 2, "REP_VOTES": 50, "DEM_VOTES": 50, "Total_Votes": 100}, index=[0]))
data_not_possible = data_not_possible.append(
    pd.DataFrame({"PRECINCT": "4", "District": 2, "REP_VOTES": 50, "DEM_VOTES": 50, "Total_Votes": 100}, index=[0]))
data_not_possible.reset_index(inplace=True)
data_not_possible.drop('index', axis=1, inplace=True)

# Test the function with the synthetic data where gerrymandering is not possible
LetsRun = isGerrymanderPossible(data_not_possible)

if LetsRun:
    print("Gerrymandering is possible")
else:
    print("Gerrymandering is not possible")


Gerrymandering solution found!
District 1: 105 Republican votes
District 2: 112 Republican votes

Precinct assignments:
District 1: ['93', '94']
District 2: ['92', '95']
GerryMandering is possible


Gerrymandering solution found!
District 1: 105 Republican votes
District 2: 105 Republican votes

Precinct assignments:
District 1: ['1', '4']
District 2: ['2', '3']
Gerrymandering is possible

Gerrymandering is not possible


# 6) Real-world Data Trials (15 pts) 





There are voter data from 5 states available herein: Alaska, Arizona, Kentucky, North Carolina, and Rhode Island. For this question you are asked to analyze Arizona and Kentucky Data. 

Note: In the example below the data is "preprocessed" to match our assumptions and downsized for reasonable experimental runtimes. 

### Notes about the tables

The create statements are stored in scripts in github including tables.sql.

Two tables in the schema:  

*  Precinct:  Holds all data for precincts, districts, and number of voter registrations by party.  There is a row for every party in each precinct, so precinct is not a unique key.  Additionally, within states, precinct is not unique, it must be used with district.

* Party:  An id and party name, just to keep the party data consistent within our database - party names and abbreviations change between states, but here we want them to be consistent.  Party can be joined with precinct on precinct.party = party.id


In [383]:
## Build the table structure
## We have two tables:  party and precinct

## The github url for the tables script
create_tables = 'https://raw.githubusercontent.com/boltonvandy/gerrymander/main/State_Data/tables.sql'

## GET contents of the tables.sql script from github
dat = gitread.request("GET", create_tables)

## Execute the table creation commands 
cursor.executescript(dat.data.decode("utf-8"))

## Preprocess for algorithm to use
view_def = ''' 
CREATE VIEW for_algo AS
SELECT * FROM
((SELECT STATE, PRECINCT, DISTRICT, VOTERS as REP_VOTES
FROM precinct WHERE PARTY = 'REP') NATURAL JOIN (
SELECT STATE, PRECINCT, DISTRICT, SUM(VOTERS) as Total_Votes
FROM precinct
WHERE (PARTY = 'REP' OR PARTY = 'DEM') 
GROUP BY STATE, PRECINCT, DISTRICT))
'''

cursor.execute(view_def)

## Commit Schema Changes
conn.commit()

## Confirm the names of the tables we built
ourtables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

if ourtables:
    print('\nTables in the Gerrymander Database\n')
    for atable in ourtables:
        print("\t" + atable[0])


Tables in the Gerrymander Database

	precinct
	party


##Example usage: Arizona

Here,the data from Arizona is loaded into the database.  

[Original Arizona Data on Kaggle](https://www.kaggle.com/arizonaSecofState/arizona-voter-registration-by-precinct)

In [384]:
## Arizona
cursor.execute("DELETE FROM precinct WHERE STATE = 'AZ'")
conn.commit()

az_url = 'https://raw.githubusercontent.com/boltonvandy/gerrymander/main/State_Data/az/az.insert.sql'

## GET contents of the script from a github url 
dat = gitread.request("GET", az_url)

## INSERT Data using statements from the github insert script
cursor.executescript(dat.data.decode("utf-8"))
conn.commit()

## Quick verification that data was loaded for this state
cursor.execute("SELECT count(*) from precinct")
verify = cursor.fetchone()[0]

cursor.execute("SELECT sum(voters), party from precinct where state = 'AZ' group by party order by 1 DESC")
print(verify, cursor.fetchall())

7270 [(1308384, 'REP'), (1251984, 'OTH'), (1169259, 'DEM'), (32096, 'LBT'), (6535, 'GRN')]


## 6a) Arizona Districts 1,2,&3   (5 out of 15 pts)

In this example, assume Districts 1/2 and 2/3 are neighboring and that precincts can be reassigned between them. Confirm (both using your code and manually) that Gerrymandering is possible between districts 2 & 3, but not 1 & 2 (given the preprocessing steps, assumptions, and downsampling done below). For the former, what is the Precinct breakdown? Your answer should be shown as code output. 


In [385]:
def fetch_and_prepare_arizona_data(district1, district2):
    """
    Fetch and prepare data for two Arizona districts.
    """
    # Fetch data for the first district
    sql_d1 = f'''
    SELECT * from for_algo where state = 'AZ' AND (DISTRICT = {district1})
    '''
    district1_data = pd.read_sql_query(sql_d1, conn).head(4)

    # Fetch data for the second district
    sql_d2 = f'''
    SELECT * from for_algo where state = 'AZ' AND (DISTRICT = {district2})
    '''
    district2_data = pd.read_sql_query(sql_d2, conn).head(4)

    # Combine data from both districts
    combined_data = district1_data.append(district2_data)
    combined_data = combined_data.reset_index(drop=True)

    # Rescale data
    combined_data["REP_VOTES"] = combined_data["REP_VOTES"] / combined_data["Total_Votes"]
    combined_data["REP_VOTES"] = pd.Series(
        [math.ceil(combined_data["REP_VOTES"][x] * 100) for x in range(len(combined_data.index))]
    )
    combined_data["Total_Votes"] = pd.Series([100 for _ in range(len(combined_data.index))])

    return combined_data

def analyze_arizona_districts(district1, district2):
    """
    Analyze if gerrymandering is possible between two Arizona districts.
    """
    data = fetch_and_prepare_arizona_data(district1, district2)
    print(f"\nAnalyzing Districts {district1} and {district2}:")
    print(data)
    print(f"\nTesting if Districts {district1} and {district2} can be gerrymandered:")
    if isGerrymanderPossible(data):
        print(f"Gerrymandering IS possible between Districts {district1} and {district2}")
    else:
        print(f"Gerrymandering is NOT possible between Districts {district1} and {district2}")

def run_arizona_analysis():
    """
    Run analysis for specified Arizona districts.
    """
    # Analyze districts 1 and 2
    analyze_arizona_districts(1, 2)
    # Analyze districts 2 and 3
    analyze_arizona_districts(2, 3)

# Run the analysis
run_arizona_analysis()


Analyzing Districts 1 and 2:
  STATE PRECINCT DISTRICT  REP_VOTES  Total_Votes
0    AZ   AP0002        1         75          100
1    AZ   AP0003        1         16          100
2    AZ   AP0005        1         18          100
3    AZ   AP0009        1         79          100
4    AZ   CH0001        2         65          100
5    AZ   CH0002        2         75          100
6    AZ   CH0003        2         63          100
7    AZ   CH0004        2         18          100

Testing if Districts 1 and 2 can be gerrymandered:
Gerrymandering is NOT possible between Districts 1 and 2

Analyzing Districts 2 and 3:
  STATE PRECINCT DISTRICT  REP_VOTES  Total_Votes
0    AZ   CH0001        2         65          100
1    AZ   CH0002        2         75          100
2    AZ   CH0003        2         63          100
3    AZ   CH0004        2         18          100
4    AZ   MC0016        3         36          100
5    AZ   MC0029        3         76          100
6    AZ   MC0037        3      

### 6b) Kentucky Districts   (10 out of 15 pts)

In this example, find two districts that are gerrymanderable and two that are not. Perform similar preprocessing steps as done in the Arizona data set, eg select 4 precincts, downsample and rescale. Confirm both district pairs using your code and manually. For the district pair that is gerrymanderable, what is the Precinct breakdown? Your answer should be shown as code output. 


In [386]:
## Kentucky!
# NOTE: the Kentucky Districts are stored as Strings. Be sure to build your query correctly :)
# See here: https://github.com/boltonvandy/gerrymander/tree/main/State_Data

cursor.execute("DELETE FROM precinct WHERE STATE = 'KY'")
conn.commit()

ky_url = 'https://raw.githubusercontent.com/boltonvandy/gerrymander/main/State_Data/ky/ky.insert.sql'

## GET contents of the script from a github url 
dat = gitread.request("GET", ky_url)

## INSERT Data using statements from the github insert script
cursor.executescript(dat.data.decode("utf-8"))
conn.commit()

## Quick verification that data was loaded for this state
cursor.execute("SELECT count(*) from precinct")
verify = cursor.fetchone()[0]

cursor.execute("SELECT sum(voters), party from precinct where state = 'KY' group by party order by 1 DESC")
print(verify, cursor.fetchall())

40498 [(1649790, 'DEM'), (1576259, 'REP'), (184839, 'OTH'), (131242, 'IND'), (14326, 'LBT'), (2014, 'GRN'), (1012, 'CONST'), (322, 'SOCWK'), (157, 'REFORM')]


In [387]:
def fetch_and_prepare_kentucky_data(district1, district2):
    """
    Fetch and prepare data for two districts.
    """
    # Fetch data for the first district
    sql_d1 = f'''
    SELECT DISTINCT p1.PRECINCT, 
           (SELECT VOTERS FROM precinct p2 
            WHERE p2.PRECINCT = p1.PRECINCT 
            AND p2.PARTY = 'REP') as REP_VOTES,
           SUM(p1.VOTERS) as Total_Votes
    FROM precinct p1
    WHERE p1.STATE = 'KY' 
    AND p1.DISTRICT LIKE '{district1}%'
    AND (p1.PARTY = 'REP' OR p1.PARTY = 'DEM')
    GROUP BY p1.PRECINCT
    LIMIT 4
    '''
    district1_data = pd.read_sql_query(sql_d1, conn)

    # Fetch data for the second district
    sql_d2 = f'''
    SELECT DISTINCT p1.PRECINCT, 
           (SELECT VOTERS FROM precinct p2 
            WHERE p2.PRECINCT = p1.PRECINCT 
            AND p2.PARTY = 'REP') as REP_VOTES,
           SUM(p1.VOTERS) as Total_Votes
    FROM precinct p1
    WHERE p1.STATE = 'KY' 
    AND p1.DISTRICT LIKE '{district2}%'
    AND (p1.PARTY = 'REP' OR p1.PARTY = 'DEM')
    GROUP BY p1.PRECINCT
    LIMIT 4
    '''
    district2_data = pd.read_sql_query(sql_d2, conn)

    # Combine data from both districts
    combined_data = district1_data.append(district2_data)
    combined_data = combined_data.reset_index(drop=True)

    # Scale the data
    combined_data["REP_VOTES"] = combined_data["REP_VOTES"] / combined_data["Total_Votes"]
    combined_data["REP_VOTES"] = pd.Series(
        [math.ceil(combined_data["REP_VOTES"][x] * 100) for x in range(len(combined_data.index))]
    )
    combined_data["Total_Votes"] = pd.Series([100 for _ in range(len(combined_data.index))])

    return combined_data

def analyze_kentucky_districts(district1, district2):
    """
    Analyze if gerrymandering is possible between two districts.
    """
    data = fetch_and_prepare_kentucky_data(district1, district2)
    print(f"\nDistricts {district1} and {district2} Data:")
    print(data)
    print(f"\nTesting if Districts {district1} and {district2} can be gerrymandered:")
    if isGerrymanderPossible(data):
        print(f"Gerrymandering IS possible between Districts {district1} and {district2}")
    else:
        print(f"Gerrymandering is NOT possible between Districts {district1} and {district2}")

def run_kentucky_analysis():
    """
    Run analysis for Kentucky districts.
    """
    # Analyze districts 1 and 2
    analyze_kentucky_districts('1', '2')
    # Analyze districts 3 and 4
    analyze_kentucky_districts('3', '4')

# Run the analysis
run_kentucky_analysis()


Districts 1 and 2 Data:
  PRECINCT  REP_VOTES  Total_Votes
0     A001         61          100
1     A002        115          100
2     A003         32          100
3     A004        242          100
4     A001         51          100
5     A002         41          100
6     A003         37          100
7     A004         44          100

Testing if Districts 1 and 2 can be gerrymandered:

Gerrymandering solution found!
District 1: 225 Republican votes
District 2: 398 Republican votes

Precinct assignments:
District 1: ['A002', 'A003', 'A002', 'A003']
District 2: ['A001', 'A004', 'A001', 'A004']
Gerrymandering IS possible between Districts 1 and 2

Districts 3 and 4 Data:
  PRECINCT  REP_VOTES  Total_Votes
0     A105         27          100
1     A107         55          100
2     A108         74          100
3     A111         97          100
4     A101          6          100
5     A102          4          100
6     A103          1          100
7     A104          8          100

Tes